Questions:
* When handling categorical features. Do I need to use pd.factorize? see  https://www.kaggle.com/shivamb/homecreditrisk-extensive-eda-baseline-0-772. I'm just using get_dummies() directly.
A: You can use both, you have to what works best for your dataset. But both can work
* When using simple imputer, I lose the column names. It turns the pd.DataFrame into a np.ndarray. Is this the right process, or are there other ways to handle missing data, NaN, Inf, numbers too big for INT32, etc?
A: Consider 'pd.fillna()'. https://pandas.pydata.org/pandas-docs/stable/missing_data.html
* Should I do one-hot encoding before or after merging the different datasets (ex, application_train & previous_application)?
A: Merger, one-hot encoding fitting the model
* For previous_application after I run the simple imputer I don't have any NaN in the np ndarray, but when I turn it into a pandas DataFrame it has 11109336. I need to turn it into a pandas DataFrame to merge it with application_train. What can I do?
A: Question is not clear

June 7 through June 17
Future steps - Must do:
* Using the other data files (Feature Engineering, Create flat dataset)
* Data visualization
* Normalizing data
* Handling skewed data
Report:
* Preliminary results

June 18 through July 1
Report:
* WEEK1: I should have a skeleton of all the areas of the report to share with mentor.
* WEEK2: I should have a first full version of the report to share with mentor and ready to submit
Future steps - Nice to do:
* CREATE A scikit-learn Pipeline
* MAKE AN ENSEMBLE METHOD OF MULTIPLE MODELS
* Measuring model/learner performance using other evaluation metric (LR+, f1, r2)?
* Hyperparameter tunening (different values until I find a max, use higher percentage of training & validation data)
* Review Handling missing or invalid data, other better methods?

DONE:
* Handling missing or invalid data
* Split train & validation set
* Hyperparameter tunening
* Measuring model/learner performance using evaluation metric (auc_roc_score)

Preliminary results:
- Categorical values turned into numerical features with one-hot encoding scheme
- Fill missing or wrong values
- RFR Default values, only application training data, score: 0.591
- RFR Best values with GridSearchCV, only application training data, score: 0.722 <- 0.62
- RFR Best values with GridSearchCV, ALL available data, score: ?
- ???? Best values with GridSearchCV, ALL available data, score: ?
- Normalizing Data
- Unskewing Data

Abbreviations:
RFR: Random Forest Regressor
????: Another regressor

# **Sections:**
[1. Import libraries & support functions](#import)  
[2. Dataset preparation](#data_prep)  
[3. Exploratory Data Analysis (EDA)](#eda)  
&nbsp; [3.1 Application Train](#eda_app_train)  
[4. Feature Engineering](#feat_eng)  
&nbsp; [4.1 Create a Flat Dataset](#flat_dataset)  
&nbsp; [4.2 Handle Skewed Continuous Data](#4.1)  
&nbsp; [4.3 Normalize Continuous Data](#4.2)  
&nbsp; [4.4 Handle Categorical Variables](#4.3)  
&nbsp; [4.5 Handle Missing Data](#4.4)  
&nbsp;&nbsp; [4.5.1 Previous Applications](#4.5)  
[5. Split Data into Training and Validation](#5)  
[6. Hyperparameter Tuning](#6)  
[7. Model Fitting & Prediction](#7)  

In [83]:
import numpy as np # linear algebra
import pandas as pd # data processing, CSVfile I/O (e.g. pd.read_csv)
import os
from plotly.offline import init_notebook_mode, iplot
import matplotlib.pyplot as plt
import plotly.graph_objs as go
from plotly import tools
# http://scikit-learn.org/stable/modules/generated/sklearn.ensemble.RandomForestRegressor.html
from sklearn.ensemble import RandomForestRegressor
# Add evaluation metric to measure the model's performance
# Regression metrics available:
# http://scikit-learn.org/stable/modules/classes.html#regression-metrics
# http://scikit-learn.org/stable/modules/model_evaluation.html#regression-metrics
# http://scikit-learn.org/stable/modules/model_evaluation.html#receiver-operating-characteristic-roc
# http://scikit-learn.org/stable/auto_examples/model_selection/plot_roc.html
# http://scikit-learn.org/stable/modules/generated/sklearn.metrics.r2_score.html
# Cannot use sklearn.metrics.accuracy_score as it is a Classification metric
from sklearn.metrics import make_scorer, r2_score, roc_auc_score
from sklearn.model_selection import GridSearchCV
from time import time
from IPython.display import display # Allows the use of display() for DataFrames

In [84]:
# Enable Debugging while I test things
debugging = False # True or False

In [85]:
# Support functions
'''
def bar_hor(df, col, title, color, w=None, h=None, lm=0, limit=100, return_trace=False, rev=False, xlb = False):
    cnt_srs = df[col].value_counts()
    yy = cnt_srs.head(limit).index[::-1] 
    xx = cnt_srs.head(limit).values[::-1] 
    if rev:
        yy = cnt_srs.tail(limit).index[::-1] 
        xx = cnt_srs.tail(limit).values[::-1] 
    if xlb:
        trace = go.Bar(y=xlb, x=xx, orientation = 'h', marker=dict(color=color))
    else:
        trace = go.Bar(y=yy, x=xx, orientation = 'h', marker=dict(color=color))
    if return_trace:
        return trace 
    layout = dict(title=title, margin=dict(l=lm), width=w, height=h)
    data = [trace]
    fig = go.Figure(data=data, layout=layout)
    iplot(fig)

def gp(col, title):
    df1 = data_train[data_train["TARGET"] == 1]
    df0 = data_train[data_train["TARGET"] == 0]
    a1 = df1[col].value_counts()
    b1 = df0[col].value_counts()
    
    total = dict(data_train[col].value_counts())
    x0 = a1.index
    x1 = b1.index
    
    y0 = [float(x)*100 / total[x0[i]] for i,x in enumerate(a1.values)]
    y1 = [float(x)*100 / total[x1[i]] for i,x in enumerate(b1.values)]

    trace1 = go.Bar(x=a1.index, y=y0, name='Target : 1', marker=dict(color="#96D38C"))
    trace2 = go.Bar(x=b1.index, y=y1, name='Target : 0', marker=dict(color="#FEBFB3"))
    return trace1, trace2 
'''

'\ndef bar_hor(df, col, title, color, w=None, h=None, lm=0, limit=100, return_trace=False, rev=False, xlb = False):\n    cnt_srs = df[col].value_counts()\n    yy = cnt_srs.head(limit).index[::-1] \n    xx = cnt_srs.head(limit).values[::-1] \n    if rev:\n        yy = cnt_srs.tail(limit).index[::-1] \n        xx = cnt_srs.tail(limit).values[::-1] \n    if xlb:\n        trace = go.Bar(y=xlb, x=xx, orientation = \'h\', marker=dict(color=color))\n    else:\n        trace = go.Bar(y=yy, x=xx, orientation = \'h\', marker=dict(color=color))\n    if return_trace:\n        return trace \n    layout = dict(title=title, margin=dict(l=lm), width=w, height=h)\n    data = [trace]\n    fig = go.Figure(data=data, layout=layout)\n    iplot(fig)\n\ndef gp(col, title):\n    df1 = data_train[data_train["TARGET"] == 1]\n    df0 = data_train[data_train["TARGET"] == 0]\n    a1 = df1[col].value_counts()\n    b1 = df0[col].value_counts()\n    \n    total = dict(data_train[col].value_counts())\n    x0 = a1.inde

In [86]:
# List available data files
#print(os.listdir("../input"))
print("Loading data files...")
# Load the application training dataset application_train.csv
data_train = pd.read_csv("../input/application_train.csv")
# Load the application testing dataset application_test.csv
data_test = pd.read_csv("../input/application_test.csv")
# Load the previous applications dataset
prev_app = pd.read_csv("../input/previous_application.csv")
print("Finished loading data files")

Loading data files...
Finished loading data files


In [87]:
'''
# Display the first 5 records of application_train.csv
display(data_train.head())
# Display the first 5 records of application_test.csv
display(data_test.head())
# See the first 5 rows of the dataframe
display(prev_app.head())

# DataFrame statistics summary for selected columns
data_train[["AMT_INCOME_TOTAL", "AMT_CREDIT", "AMT_ANNUITY", "AMT_GOODS_PRICE"]].describe()
#print(len(data_train.columns))
'''

'\n# Display the first 5 records of application_train.csv\ndisplay(data_train.head())\n# Display the first 5 records of application_test.csv\ndisplay(data_test.head())\n# See the first 5 rows of the dataframe\ndisplay(prev_app.head())\n\n# DataFrame statistics summary for selected columns\ndata_train[["AMT_INCOME_TOTAL", "AMT_CREDIT", "AMT_ANNUITY", "AMT_GOODS_PRICE"]].describe()\n#print(len(data_train.columns))\n'

In [88]:
# Total number of records
print("Total number of records in the train dataset: {:,}".format(len(data_train)))
# Total number of features. Excluding the load ids (SK_ID_CURR) and the target variable (TARGET).
print("Total number of features in the train dataset: {}".format(data_train.shape[1] - 2))

# Total number of records
print("Total number of records in the test dataset: {:,}".format(len(data_test)))
# Total number of features. Excluding the load ids (SK_ID_CURR). There is NO target variable (TARGET) in the test dataset.
print("Total number of features in the test dataset: {}".format(data_test.shape[1] - 1))

# Total number of features. Excluding the current loan ids (SK_ID_CURR) and the previous loan ids (SK_ID_PREV)
print("Total number of features of previous applications, before one-hot encoding: {}".format(prev_app.shape[1] - 2))
print("Previous applications has {:,} samples".format(prev_app.shape[0]))
# Check if therer is any NaN value in the dataset
print("Total number of NaN in the dataframe: {:,}".format(prev_app.isnull().sum().sum())) # TODO: REMOVE

Total number of records in the train dataset: 307,511
Total number of features in the train dataset: 120
Total number of records in the test dataset: 48,744
Total number of features in the test dataset: 120
Total number of features of previous applications, before one-hot encoding: 35
Previous applications has 1,670,214 samples
Total number of NaN in the dataframe: 11,109,336


In [89]:
# GRAPHS ARE NOT DISPLAYING
'''
# Target Variable Distribution 
bar_hor(data_train, "TARGET", "Distribution of Target Variable" , ["#44ff54", '#ff4444'], h=350, w=600, lm=200, xlb = ['Target : 1','Target : 0'])

tr0 = bar_hor(data_train, "CODE_GENDER", "Distribution of CODE_GENDER Variable" ,"#f975ae", w=700, lm=100, return_trace= True)
tr1, tr2 = gp('CODE_GENDER', 'Distribution of Target with Applicant Gender')

fig = tools.make_subplots(rows=1, cols=3, print_grid=False, subplot_titles = ["Gender Distribution" , "Gender, Target=1" ,"Gender, Target=0"])
fig.append_trace(tr0, 1, 1);
fig.append_trace(tr1, 1, 2);
fig.append_trace(tr2, 1, 3);
fig['layout'].update(height=350, showlegend=False, margin=dict(l=50));
iplot(fig);
'''

'\n# Target Variable Distribution \nbar_hor(data_train, "TARGET", "Distribution of Target Variable" , ["#44ff54", \'#ff4444\'], h=350, w=600, lm=200, xlb = [\'Target : 1\',\'Target : 0\'])\n\ntr0 = bar_hor(data_train, "CODE_GENDER", "Distribution of CODE_GENDER Variable" ,"#f975ae", w=700, lm=100, return_trace= True)\ntr1, tr2 = gp(\'CODE_GENDER\', \'Distribution of Target with Applicant Gender\')\n\nfig = tools.make_subplots(rows=1, cols=3, print_grid=False, subplot_titles = ["Gender Distribution" , "Gender, Target=1" ,"Gender, Target=0"])\nfig.append_trace(tr0, 1, 1);\nfig.append_trace(tr1, 1, 2);\nfig.append_trace(tr2, 1, 3);\nfig[\'layout\'].update(height=350, showlegend=False, margin=dict(l=50));\niplot(fig);\n'

## <a id="4.1">4.1 Feature Engineering - Create a Flat Dataset</a>

### <a id="4.1.1">4.1.1 Feature Engineering - Create a Flat Dataset - Previous Applications</a>

In [90]:
'''
# Check if therer is any NaN value in the pd dataframe df
df.isnull().sum().sum()

# Check if therer is any NaN value in the np nparray np_array
np.argwhere(np.isnan(np_array))
'''

'\n# Check if therer is any NaN value in the pd dataframe df\ndf.isnull().sum().sum()\n\n# Check if therer is any NaN value in the np nparray np_array\nnp.argwhere(np.isnan(np_array))\n'

In [91]:
if debugging:
    display(prev_app.shape)
    display(prev_app.head())
    display(prev_app[prev_app['SK_ID_CURR'] == 271877])

In [92]:
'''
#### Q: WHY IS THIS NEEDED???
# count the number of previous applications for a given 'SK_ID_CURR'
prev_app_count = prev_app[['SK_ID_CURR', 'SK_ID_PREV']].groupby('SK_ID_CURR').count()
prev_app['SK_ID_PREV'] = prev_app['SK_ID_CURR'].map(prev_app_count['SK_ID_PREV'])
'''

"\n#### Q: WHY IS THIS NEEDED???\n# count the number of previous applications for a given 'SK_ID_CURR'\nprev_app_count = prev_app[['SK_ID_CURR', 'SK_ID_PREV']].groupby('SK_ID_CURR').count()\nprev_app['SK_ID_PREV'] = prev_app['SK_ID_CURR'].map(prev_app_count['SK_ID_PREV'])\n"

In [93]:
if debugging:
    display(prev_apps_count.head())

In [94]:
if debugging:
    display(data_train.shape)
    display(data_train.head())

In [95]:
### Q: ALL THE CATEGORICAL FEATURES ARE LOST BY DOING THE MEAN
### TODO: ALL THE CATEGORICAL FEATURES ARE LOST BY DOING THE MEAN
## Average values for all other features in previous applications
prev_app_avg = prev_app.groupby('SK_ID_CURR').mean()
prev_app_avg.columns = ['p_' + col for col in prev_app_avg.columns]
data_train = data_train.merge(right=prev_app_avg.reset_index(), how='left', on='SK_ID_CURR')

In [96]:
if debugging:
    display(prev_app_avg.shape)
    display(prev_app_avg.head())
    #display(prev_app_avg[prev_app_avg['SK_ID_CURR'] == 271877].head())
    display(data_train.shape)
    display(data_train['p_AMT_ANNUITY'].head())
    print("Total number of NaN in the data dataframe: {:,}".format(data.isnull().sum().sum()))

In [97]:
# Transforming skewed continuous features

In [98]:
# Normalizing numerical features
#from sklearn.preprocessing import MinMaxScaler

In [99]:
# Turn categorical variables into numerical features using the one-hot encoding scheme

# Total number of features. Excluding the load ids (SK_ID_CURR) and the target variable (TARGET)
print("Total number of features of train data, before one-hot encoding: {}".format(data_train.shape[1] - 2))
# Total number of features. Excluding the load ids (SK_ID_CURR). The test dataset has NO target label (TARGET)
print("Total number of features of test data, before one-hot encoding: {}".format(data_test.shape[1] - 1))

# One-hot encoding
data_train_encoded = pd.get_dummies(data_train)
data_test_encoded = pd.get_dummies(data_test)

# Total number of features. Excluding the loan ids (SK_ID_CURR) and the target variable (TARGET)
print("Total number of features of train data, after one-hot encoding: {}".format(data_train_encoded.shape[1] - 2))
# Total number of features. Excluding the loan ids (SK_ID_CURR). The test dataset has NO target label (TARGET)
print("Total number of features of test data, after one-hot encoding: {}".format(data_test_encoded.shape[1] - 1))

# New list of features
#print(list(data_train_encoded.columns))

# Determine what columns are missing
train_list = list(data_train_encoded.columns)
test_list = list(data_test_encoded.columns)
difference = [e for e in train_list if e not in test_list]
print(difference)

# Add those columns to the test set will all zeros
data_test_encoded_complete = data_test_encoded
for e in difference:
    if e != 'TARGET':
        data_test_encoded_complete[e] = 0

print("Total number of features of test data, after one-hot encoding: {}".format(data_test_encoded_complete.shape[1] - 1))

Total number of features of train data, before one-hot encoding: 140
Total number of features of test data, before one-hot encoding: 120
Total number of features of train data, after one-hot encoding: 264
Total number of features of test data, after one-hot encoding: 241
['TARGET', 'p_SK_ID_PREV', 'p_AMT_ANNUITY', 'p_AMT_APPLICATION', 'p_AMT_CREDIT', 'p_AMT_DOWN_PAYMENT', 'p_AMT_GOODS_PRICE', 'p_HOUR_APPR_PROCESS_START', 'p_NFLAG_LAST_APPL_IN_DAY', 'p_RATE_DOWN_PAYMENT', 'p_RATE_INTEREST_PRIMARY', 'p_RATE_INTEREST_PRIVILEGED', 'p_DAYS_DECISION', 'p_SELLERPLACE_AREA', 'p_CNT_PAYMENT', 'p_DAYS_FIRST_DRAWING', 'p_DAYS_FIRST_DUE', 'p_DAYS_LAST_DUE_1ST_VERSION', 'p_DAYS_LAST_DUE', 'p_DAYS_TERMINATION', 'p_NFLAG_INSURED_ON_APPROVAL', 'CODE_GENDER_XNA', 'NAME_INCOME_TYPE_Maternity leave', 'NAME_FAMILY_STATUS_Unknown']
Total number of features of test data, after one-hot encoding: 264


In [100]:
# Separate the target label from the train dataset. The column we are interested is 'TARGET'. Name it target_train.
target_train = data_train['TARGET']
print("Training target label has {:,} samples".format(target_train.shape[0]))

# Remove target label from the train dataset and rename to features_train.
features_train = data_train_encoded.drop(['TARGET'], axis=1)

# Test data has no taget label 'TARGET' in the dataset
features_test = data_test_encoded_complete

Training target label has 307,511 samples


In [101]:
# Handle missing values
# Source: https://www.kaggle.com/dansbecker/handling-missing-values
# http://scikit-learn.org/dev/modules/generated/sklearn.impute.SimpleImputer.html

from sklearn.impute import SimpleImputer
my_imputer = SimpleImputer()
features_train = my_imputer.fit_transform(features_train)
# Use this instead:
#features_test = my_imputer.transform(features_test)
features_test = my_imputer.fit_transform(features_test)

# Consider turning features_train & features_test into a pandas DataFrame, instead of being a numpy ndarray
# prev_app_imputed = pd.DataFrame(my_imputer.fit_transform(prev_app_encoded), columns=prev_app_encoded.columns)

In [102]:
# Shuffle and split the data
# http://scikit-learn.org/stable/modules/generated/sklearn.model_selection.train_test_split.html
from sklearn.model_selection import train_test_split

# Split the 'features' and 'target label' data into training and validating sets
X_train, X_val, y_train, y_val = train_test_split(features_train,
                                                  target_train,
                                                  test_size=0.2,
                                                  random_state=42)

print("Original Training set has {:,} samples.".format(features_train.shape[0]))
print("After split Training set has {:,} samples.".format(X_train.shape[0]))
print("After split Validating set has {:,} samples.".format(X_val.shape[0]))
print("Testing set has {:,} samples.".format(data_test.shape[0]))

Original Training set has 307,511 samples.
After split Training set has 246,008 samples.
After split Validating set has 61,503 samples.
Testing set has 48,744 samples.


In [103]:
# Run GridSearchCV or fully train an estimator
run_mode = 'train_estimator' # 'grid_search' or 'train_estimator'

In [104]:
# Run GridSearchCV
if run_mode == 'grid_search':
    perc_samples = 0.001
    print("Preparing to run Hyperparameters tunning with GridSearchCV using {0:.2f}% of the training samples".format(perc_samples * 100))
    features_train_small = X_train[:int(perc_samples * X_train.shape[0])]
    target_train_small = y_train[:int(perc_samples * y_train.shape[0])]
    features_val_small = X_val[:int(perc_samples * X_val.shape[0])]
    target_val_small = y_val[:int(perc_samples * y_val.shape[0])]
    #features_test_small = features_test[:int(perc_samples * features_test.shape[0])]

    # Initialize the Estimator (Learner or Regression Model)
    estimator = RandomForestRegressor(n_jobs=-1,
                                      random_state=42,
                                      verbose=0)

    # Determine which Parameters to tune
    '''
    Tested so far:
    parameters = {
        'n_estimators': [9, 10, 11, 12, 13, 14, 15],
        'criterion': ['mse', 'mae'],
        'min_samples_leaf': [1, 2, 3, 4, 5, 6, 7],
        'max_features': [0.01, 0.1, 0.25, 0.45, 0.5, 0.55, 0.6, 0.75],
        'min_samples_split': [2, 3, 4, 5],
        'warm_start': [False, True]
    }
    '''
    parameters = {
        'n_estimators': [115, 125, 135],
        'min_samples_leaf': [62, 75, 87],
        'max_features': [0.18, 0.2, 0.23],
        'min_samples_split': [2, 3],
    }

    # Create a scorer to measure hyperparameters performance
    scorer = make_scorer(roc_auc_score)

    # Create GridSearchCV grid object
    grid_obj = GridSearchCV(estimator=estimator, 
                            param_grid=parameters, 
                            scoring=scorer)

    # Fit the GridSearchCV grid object with the reduced training dataset and find the best hyperparameters
    start = time()
    grid_fit = grid_obj.fit(features_train_small, target_train_small)
    end = time()
    grid_fit_time = (end - start) / 60 # Ellapsed time in minutes
    print("GridSearchCV estimator fit time: {0:.2f} minutes".format((end - start) / 60))

    # Get the best estimator
    best_est = grid_obj.best_estimator_
    print("Best Estimator: \n{}\n".format(best_est))

    # Get the best score
    best_score = grid_obj.best_score_
    print("Best Estimator Score: {}\n".format(best_score))

    # Get the best parameters
    best_params = grid_obj.best_params_
    print("Best Hyperparameters that yield the best score: \n{}\n".format(best_params))

    # Make predictions with unoptimized estimator on the validation set
    pred_val = (estimator.fit(features_train_small, target_train_small)).predict(features_val_small)
    print("Unoptimized Estimator prediction score on Validation set: \t{}".format(roc_auc_score(target_val_small, pred_val)))

    # Predict with the best estimator on the validation set
    best_pred_val = best_est.predict(features_val_small)
    print("Optimized Estimator prediction score on Validation set: \t{}".format(roc_auc_score(target_val_small, best_pred_val)))

    # Predict with the best estimator on the testing set
    #pred_test = best_est.predict(features_test)

In [105]:
# Train estimator
# TODO: rename or remove '_small', it might be '_full' or nothing
if run_mode == 'train_estimator':
    # Use the full training and validation datasets to fit the estimator with the best hyperparameters
    perc_samples = 1
    print("Preparing to train an estimator using {0:.2f}% of the training dataset".format(perc_samples * 100))
    features_train_small = X_train[:int(perc_samples * X_train.shape[0])]
    target_train_small = y_train[:int(perc_samples * y_train.shape[0])]
    features_val_small = X_val[:int(perc_samples * X_val.shape[0])]
    target_val_small = y_val[:int(perc_samples * y_val.shape[0])]
    features_test_small = features_test[:int(perc_samples * features_test.shape[0])]

    # Initialize the Estimator (Learner or Regression Model) with the best hyperparameters
    '''
    estimator = RandomForestRegressor(criterion='mae', # default='mse', VERY SLOW
                                      min_samples_split=2, # default=2
                                      warm_start=False) # default=False
    Best Performance:
    estimator = RandomForestRegressor(n_estimators=125, # default=10
                                      max_features=0.2, # default='auto'
                                      min_samples_split=2, # default=2
                                      min_samples_leaf=75, # default=1
                                      n_jobs=-1, # default=1
                                      random_state=42, # default=None
                                      verbose=2) # default=0
                                      ROC_AUC_SCORE:  0.7417496532130599 (% samples: 0.15)
                                      LEADERBOARD SCORE: 0.722 (NO improvement)

    estimator = RandomForestRegressor(n_estimators=100, # default=10
                                      max_features=0.2, # default='auto'
                                      min_samples_split=3, # default=2
                                      min_samples_leaf=50, # default=1
                                      n_jobs=-1, # default=1
                                      random_state=42, # default=None
                                      verbose=2) # default=0
                                      ROC_AUC_SCORE:  0.7286786065442892 (% samples: 0.1)
                                      LEADERBOARD SCORE: 0.722

    estimator = RandomForestRegressor(n_estimators=12, # default=10
                                      max_features=0.45, # default='auto'
                                      min_samples_leaf=2, # default=1
                                      n_jobs=-1, # default=1
                                      random_state=42, # default=None
                                      verbose=2) # default=0
                                      LEADERBOARD SCORE: 0.62
    '''
    estimator = RandomForestRegressor(n_estimators=125, # default=10
                                      max_features=0.2, # default='auto'
                                      min_samples_split=2, # default=2
                                      min_samples_leaf=75, # default=1
                                      n_jobs=-1, # default=1
                                      random_state=42, # default=None
                                      verbose=0) # default=0

    # Fit the estimator with the training dataset
    start = time()
    estimator.fit(features_train_small, target_train_small)
    end = time()
    print("Estimator fit time: {0:.2f} minutes".format((end - start) / 60))

    # Predict with the validation dataset
    pred_val = estimator.predict(features_val_small)
    print("Estimator prediction score on Validation set: \t{}".format(roc_auc_score(target_val_small, pred_val)))
    
    # Predict using the 'test' dataset for submission
    pred_test = estimator.predict(features_test_small)
    #pred_test = estimator.predict(features_test)

    # Prepare prediction for submission
    submission = pd.DataFrame()
    # Need to replace data_test_encoded_complete
    submission['SK_ID_CURR'] = data_test_encoded_complete['SK_ID_CURR'][:int(perc_samples * features_test.shape[0])]
    submission['TARGET'] = pred_test
    submission.head()
    submission.to_csv('RFR.csv', index=False)

Preparing to train an estimator using 100.00% of the training dataset
Estimator fit time: 4.02 minutes
Estimator prediction score on Validation set: 	0.7542736992401178
